# "[ML] Ensemble - Boosting(LightGBM)"
> "신용거래 사기 탐지"

- toc: true
- branch: master
- badges: true
- comments: true
- categories: [Ensemble,Boosting,LightGBM]
- author: 도형준

# 신용거래 사기 탐지

## 데이터 적재

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 원본 데이터: https://www.kaggle.com/datasets/kartik2112/fraud-detection
file_url = 'https://media.githubusercontent.com/media/musthave-ML10/data_source/main/fraud.csv'
df = pd.read_csv(file_url)
df.head()

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [3]:
df.info() # 결측치나 카운트에 대한 정보가 없음.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1852394 entries, 0 to 1852393
Data columns (total 22 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   trans_date_trans_time  object 
 1   cc_num                 int64  
 2   merchant               object 
 3   category               object 
 4   amt                    float64
 5   first                  object 
 6   last                   object 
 7   gender                 object 
 8   street                 object 
 9   city                   object 
 10  state                  object 
 11  zip                    int64  
 12  lat                    float64
 13  long                   float64
 14  city_pop               int64  
 15  job                    object 
 16  dob                    object 
 17  trans_num              object 
 18  unix_time              int64  
 19  merch_lat              float64
 20  merch_long             float64
 21  is_fraud               int64  
dtypes: float64(5), int

* trans_date_trans_time : 거래 시간
* cc_num : 카드 번호. 고윳값이기 때문에 여기서는 id처럼 활용할 수 있음
* merchant : 거래 상점
* category : 거래 상점의 범주(애완용품, 여행, 엔터테인먼트 등)
* amt: 거래금액 (amount)
* first / last : 이름
* gender : 성별
* street / state / zip : 고객 거주지 정보
* lat / long : 고객주소에 대한 위도 및 경도
* city_pop : 고객의 zipcode 속하는 인구 수
* job : 직업
* dob : 생년월일
* trans_num : 거래번호
* unix_time : 거래시간 (유닉스 타임스탬프)
* merch_lat / merch_long : 상점의 위경도
---
* is_fraud : 사기거래 여부 (이상거래 여부) -> 종속변수

In [4]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1852394 entries, 0 to 1852393
Data columns (total 22 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   trans_date_trans_time  1852394 non-null  object 
 1   cc_num                 1852394 non-null  int64  
 2   merchant               1852394 non-null  object 
 3   category               1852394 non-null  object 
 4   amt                    1852394 non-null  float64
 5   first                  1852394 non-null  object 
 6   last                   1852394 non-null  object 
 7   gender                 1852394 non-null  object 
 8   street                 1852394 non-null  object 
 9   city                   1852394 non-null  object 
 10  state                  1852394 non-null  object 
 11  zip                    1852394 non-null  int64  
 12  lat                    1852394 non-null  float64
 13  long                   1852394 non-null  float64
 14  city_pop          

In [5]:
pd.options.display.float_format = '{:.2f}'.format
df.describe()

,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,1852394.00,1852394.00,1852394.00,1852394.00,1852394.00,1852394.00,1852394.00,1852394.00,1852394.00,1852394.00
mean,417386038393710400.00,70.06,48813.26,38.54,-90.23,88643.67,1358674218.83,38.54,-90.23,0.01
std,1309115265318734848.00,159.25,26881.85,5.07,13.75,301487.62,18195081.39,5.11,13.76,0.07
min,60416207185.00,1.00,1257.00,20.03,-165.67,23.00,1325376018.00,19.03,-166.67,0.00
25%,180042946491150.00,9.64,26237.00,34.67,-96.80,741.00,1343016823.75,34.74,-96.90,0.00
50%,3521417320836166.00,47.45,48174.00,39.35,-87.48,2443.00,1357089331.00,39.37,-87.44,0.00
75%,4642255475285942.00,83.10,72042.00,41.94,-80.16,20328.00,1374581485.25,41.96,-80.25,0.00
max,4992346398065154048.00,28948.90,99921.00,66.69,-67.95,2906700.00,1388534374.00,67.51,-66.95,1.00


## 전처리

In [6]:
# 사용되지 않는 변수 제거
df.drop(['first', 'last', # 이름
         'street', 'city', 'state', 'zip', # 주소
         'trans_num', 'unix_time', # 거래번호 / 유닉스타임 (중복)
         'job', 'merchant' # 직업, 가게
         ], axis=1, inplace=True)

In [7]:
# 날짜 형태의 데이터 Object 문자열로 저장되어있다
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1852394 entries, 0 to 1852393
Data columns (total 12 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   trans_date_trans_time  datetime64[ns]
 1   cc_num                 int64         
 2   category               object        
 3   amt                    float64       
 4   gender                 object        
 5   lat                    float64       
 6   long                   float64       
 7   city_pop               int64         
 8   dob                    object        
 9   merch_lat              float64       
 10  merch_long             float64       
 11  is_fraud               int64         
dtypes: datetime64[ns](1), float64(5), int64(3), object(3)
memory usage: 169.6+ MB


## 피쳐 엔지니어링
- 원래 고객의 거래패턴에서 벗어나는 거래

### 결제 금액

In [8]:
# Z 점수 (정규분포) <- (x - 평균) / 표준편차
# amt_info = df.groupby('cc_num').agg(['mean', 'std'])
# cc_num : 카드번호별 그룹을 묶어서, agg -> 여러 그룹함수(여러개의 값들을 통해 계산하는 통계값)
# mean = 평균 / std = 표준편차 => amt(결제금액) => cc_num
amt_info = df.groupby('cc_num').agg(['mean', 'std']).amt.reset_index()
amt_info.head()

,cc_num,mean,std
0,60416207185,59.26,142.87
1,60422928733,65.48,92.04
2,60423098130,96.38,1000.69
3,60427851591,107.49,131.01
4,60487002085,64.10,153.21


In [9]:
# A.merge(B, on = KEY, how=WAY)
# A라는 데이터프레임에 B를 합쳐주겠다 => (index?) => cc_num
# LEFT
df = df.merge(amt_info, on = 'cc_num', how='left')

In [10]:
df.columns

Index(['trans_date_trans_time', 'cc_num', 'category', 'amt', 'gender', 'lat',
       'long', 'city_pop', 'dob', 'merch_lat', 'merch_long', 'is_fraud',
       'mean', 'std'],
      dtype='object')

In [11]:
df[['cc_num', 'amt', 'mean', 'std']].head()

,cc_num,amt,mean,std
0,2703186189652095,4.97,89.41,127.53
1,630423337322,107.23,56.08,159.20
2,38859492057661,220.11,69.92,116.69
3,3534093764340240,45.00,80.09,280.08
4,375534208663984,41.96,95.34,94.32


In [12]:
# (x - 평균)/표준편차
df['amt_z_score'] = (df['amt'] - df['mean']) / df['std']

In [13]:
df['amt_z_score'].head()

0   -0.66
1    0.32
2    1.29
3   -0.13
4   -0.57
Name: amt_z_score, dtype: float64

In [14]:
df.drop(['mean', 'std'], axis=1, inplace=True)

### 범주형 결제금액

In [15]:
# 결제를 한 사람의 카드번호 / 결제가 일어난 상점의 종류(분류)
# agg -> 그룹을 대상으로 통계값. 그룹함수
# mean : 평균 / std : 표준편차
category_info = df.groupby(['cc_num', 'category']).agg(['mean', 'std'])['amt']

In [16]:
category_info.head()

mean   std
cc_num      category                  
60416207185 entertainment  51.84 65.49
            food_dining    26.74 46.38
            gas_transport  59.78 15.76
            grocery_net    52.15 17.69
            grocery_pos   101.56 21.89

In [17]:
# cc_num, category -> merge. / mean, std => z_score / mean, std? drop
df = df.merge(category_info, on=['cc_num', 'category'], how='left')
df[['cc_num', 'category', 'amt', 'mean', 'std']].head()

,cc_num,category,amt,mean,std
0,2703186189652095,misc_net,4.97,84.86,116.07
1,630423337322,grocery_pos,107.23,99.64,23.90
2,38859492057661,entertainment,220.11,46.65,60.39
3,3534093764340240,gas_transport,45.00,61.54,15.75
4,375534208663984,misc_pos,41.96,35.48,4.93


In [18]:
df['cat_z_score'] = (df['amt'] - df['mean']) / df['std']

In [19]:
df['cat_z_score'].head()

0   -0.69
1    0.32
2    2.87
3   -1.05
4    1.31
Name: cat_z_score, dtype: float64

In [20]:
import geopy.distance

In [21]:
# coordinate 좌표 (위.경도)
# 1. 상점의 위치 (merchant)
# 2. 고객의 위치 (customer)
df['merch_coord'] = pd.Series(zip(df.merch_lat, df.merch_long))
df['cust_coord'] = pd.Series(zip(df['lat'], df['long']))

In [22]:
df['distance'] = df.apply(lambda x: geopy.distance.distance(x['merch_coord'], x['cust_coord']).km, axis=1)

In [23]:
# .km 빼먹으면 -> 단위를 포함해서 object -> object => float
# df['distance'] = df['distance'].str.split(expand=True)[0].astype('float64')

In [24]:
distance_info = df.groupby('cc_num').agg(['mean', 'std'])['distance'].reset_index()

In [ ]:
distance_info

In [25]:
df = df.merge(distance_info, on = 'cc_num', how='left')
df['distance_z_score'] = (df['distance'] - df['mean']) / df['std']

KeyError: ignored

In [ ]:
df['distance_z_score']

In [ ]:
df.drop(['mean', 'std'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df['age'] = 2021 - pd.to_datetime(df['dob']).dt.year # 만 나이

df(['cc_num', 'lat', 'long',
    'merch_lat', 'merch_long', 'dob',
    'merch_coord', 'cust_coord'], axis=1, inplace=True)

In [ ]:
df = pd.get_dummies(df, columns = ['category', 'gender'], drop_first=True)

In [ ]:
df.head()

In [ ]:
df.set_index('trans_date_trans_time', inplace=True)

In [ ]:
df.index # 2020-07-01

In [ ]:
train = df[df.index < '2020-07-01']
test = df[df.index >= '2020-07-01']

In [ ]:
X_train = train.drop('is_fraud', axis=1)
y_train = train['is_fraud']
X_test = test.drop('is_fraud', axis=1)
y_test = train['is_fraud']

In [ ]:
import lightgbm as lgb

In [ ]:
model = lgb.LGBMClassifier(random_state=100)
model.fit(X_train, y_train)
pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)